In [11]:
import numpy as np
import skimage
import matplotlib.pyplot as plt
from tempfile import TemporaryFile
from PIL import Image, ImageEnhance
from os import listdir
import imghdr
from skimage.transform import rotate, AffineTransform, warp
from skimage import filters
from torchvision import transforms
import cv2
import scipy
import random

In [ ]:
rmdir './Data_full/.ipynb_checkpoints'

In [13]:
#Load Data
# return array of images
def loadImages(path):
    imagesList = listdir(path)
    imgs = []
    labels = []
    for image in imagesList:
      if imghdr.what(path + image) == 'png':
        if (image[len(image) - 13] == '1' and image[6].isalpha()): # only add 5 of each image, only add alphabetical values
          img = Image.open(path + image)
          imgs.append(img)
          labels.append(image[6]) # assumes that filename structure is 'handx_[label]_....'
    return imgs, labels

# Convert png img array to array np arrays
def ImagesToArray(imgs):
  imgs_array = []
  for img in imgs:
    img_array = np.array(img)
    imgs_array.append(img_array)
  return imgs_array

# Reshape Images   
def shape600(x):
  reshaped_array = np.zeros((len(x), 600, 600, 3))
  for i, img in enumerate(x):
    x_pad_width = (600 - img.shape[0])//2
    y_pad_width = (600 - img.shape[1])//2
    reshaped_array[i,:,:,:] = np.pad(img, ((x_pad_width, x_pad_width + (img.shape[0])%2), (y_pad_width, y_pad_width+(img.shape[1]%2)), (0,0)), constant_values=img[0][0][0])
  return reshaped_array

# Normalize images
def Normalize(imgs):
  new_imgs = []
  for img in imgs:
      # flat_img = img.flatten()
      m = np.mean(img)
      std = np.std(img)
      img = (img-m)/std
      new_imgs.append(img)
  return new_imgs

# x is input image, sd is how much to blur
def blur(imgs, sd=1):
  filtered_img = np.zeros((imgs.shape[0], 600, 600, 3))
  for i in range(imgs.shape[0]):
    filtered_img[i,:,:,:] = skimage.filters.gaussian(imgs[i,:,:,:], sigma=sd)
    #filtered_img.save('./Filter_gaussian/img_' + i + '_gaussianfilt.png')
  return filtered_img

def Scale(imgs):
  scaled_images = np.zeros((len(imgs), 600, 600, 3))
  for i, img in enumerate(imgs):
    ratio = random.uniform(.3, .7)
    x = int(ratio * 600 / 2)
    scaled = img[x:600-x, x:600-x]
    # scaled = cv2.imread(scaled)
    res = cv2.resize(scaled, dsize=(600, 600), interpolation=cv2.INTER_CUBIC)
    scaled_images[i] = res
  return scaled_images
    # final.save('./Crop/img_' + str(i) + '_scale.png')

def Brighten_light(imgs):
  Brightened_light_images = np.zeros((len(imgs), 600, 600, 3))
  for i in range (imgs.shape[0]):
    Brightened_light_images[i] = imgs[i] + (0.2, 0.2, 0.2)
    # light_img.save('./Brightness_light/img_' + str(i) + '_light.png')
  return Brightened_light_images

def Brighten_dark(imgs):
  Brightened_dark_images = np.zeros((len(imgs), 600, 600, 3))
  for i in range (imgs.shape[0]):
    Brightened_dark_images[i] = imgs[i] - (0.2, 0.2, 0.2)
    # dark_img.save('./Brightness_dark/img_' + str(i) + '_dark.png')
  return Brightened_dark_images

def Rotate30(imgs): 
  rot30_imgs = np.empty([imgs.shape[0], imgs.shape[1], imgs.shape[2], imgs.shape[3]])
  for i, img in enumerate(imgs): 
    rand_dir = random.choice([-1, 1])
    new_img = rotate(img, rand_dir * 30) 
    rot30_imgs[i, :] = new_img
  return rot30_imgs

def VerticalFlip(imgs): 
  flip_imgs = np.empty([imgs.shape[0], imgs.shape[1], imgs.shape[2], imgs.shape[3]])
  for i, img in enumerate(imgs): 
    new_img = np.fliplr(img)
    flip_imgs[i, :, :, :] = new_img
  return flip_imgs

def Translation(imgs): 
  trans_imgs = np.empty([imgs.shape[0], imgs.shape[1], imgs.shape[2], imgs.shape[3]])
  for i, img in enumerate(imgs): 
    rand_x = random.randrange(-150, 150, 50)
    rand_y = random.randrange(-150, 150, 50)
    transform = AffineTransform(translation=(rand_x,rand_y))
    new_img = warp(img,transform, mode="constant")  
    trans_imgs[i, :] = new_img
  return trans_imgs

# load png_images
path = "./Data_full/"

# images in an array named imgs
imgs, labels = loadImages(path)

print(len(imgs))

# Step 1 convert png_images to np arrays
imgs_array_before = ImagesToArray(imgs)

def Augment(imgs_array_before, i):
  # Step 2 Normalize images
  imgs_array_normalize = Normalize(imgs_array_before)

  # Step 3 Reshape the images
  imgs_array_reshaped = shape600(imgs_array_normalize)

  # Step 4 Blur the images
  blurred_imgs = blur(imgs_array_reshaped, 10)

  # Step 5 Scale the images
  scaled_imgs = Scale(imgs_array_reshaped)

  # Step 6 Brighten the images
  light_imgs = Brighten_light(imgs_array_reshaped)
  dark_imgs = Brighten_dark(imgs_array_reshaped)

  # Step 7 Flip the images
  flipped_imgs = VerticalFlip(imgs_array_reshaped)

  # Step 8 Add translation to images
  translated_imgs = Translation(imgs_array_reshaped)

  # Step 9 Rotate 30 Degrees
  rotated30_imgs = Rotate30(imgs_array_reshaped)
  rotatedNeg30_imgs = RotateNeg30(imgs_array_reshaped)

  # Step 10 combine all augmented images to np array of shape((num_augmentations + 1) * num_images, 600, 600, 3)
  final_imgs_temp = np.concatenate((imgs_array_reshaped, blurred_imgs, light_imgs, dark_imgs, scaled_imgs, translated_imgs, flipped_imgs, rotated30_imgs, rotatedNeg30_imgs), 0)
  print(final_imgs_temp.shape)
  np.save('./FinalImages/imgs' + str(i) + '.npy', final_imgs_temp)
  # Show the progression of images for each step
  # fig, ax = plt.subplots(1, 11, figsize=(15,10))
  # ax[0].imshow(imgs_array_before[0]) # Step 1
  # ax[1].imshow(imgs_array_normalize[0]) # Step 2
  # ax[2].imshow(imgs_array_reshaped[0]) # Step 3
  # ax[3].imshow(blurred_imgs[0]) # Step 4
  # ax[4].imshow(scaled_imgs[0]) # Step 5
  # ax[5].imshow(light_imgs[0]) # Step 6
  # ax[6].imshow(dark_imgs[0]) # Step 6
  # ax[7].imshow(flipped_imgs[0]) # Step 7
  # ax[8].imshow(translated_imgs[0]) # Step 8
  # ax[9].imshow(rotated30_imgs[0]) # Step 9
  # ax[10].imshow(rotatedNeg30_imgs[0]) # Step 9
  return final_imgs_temp


130


In [10]:
final_imgs = np.empty([0, 600, 600, 3])
n = int(900/50)
for i in range(n):
  final_imgs_temp = Augment(imgs_array_before[i * 50:(i + 1) * 50], i)
  final_imgs = np.concatenate((final_imgs, final_imgs_temp), 0)

print(final_imgs.shape)
np.save('./FinalImages/imgs.npy', final_imgs)

/Users/shreyawadhwa/miniconda3/envs/pytorch_p37/lib/python3.8/site-packages/skimage/_shared/utils.py:348: RuntimeWarning: Images with dimensions (M, N, 3) are interpreted as 2D+RGB by default. Use `multichannel=False` to interpret as 3D image with last dimension of length 3.
  return func(*args, **kwargs)


(450, 600, 600, 3)
(450, 600, 600, 3)
(450, 600, 600, 3)
(450, 600, 600, 3)
(450, 600, 600, 3)
(450, 600, 600, 3)
(450, 600, 600, 3)
(450, 600, 600, 3)
(450, 600, 600, 3)
(450, 600, 600, 3)
(450, 600, 600, 3)
(450, 600, 600, 3)


OSError: 486000000 requested and 41156608 written